In [1]:
import os
# Change this path depending on where you're running
# path = "/Users/agni/Packages/partners-cerebral-edema/Glove/"
# os.chdir(path)
print(os.getcwd())

/Users/agni/Packages/partners-cerebral-edema/Glove


In [2]:
path = "/Users/agni/Packages/partners-cerebral-edema/Glove/"
os.chdir(path)
print(os.getcwd())

/Users/agni/Packages/partners-cerebral-edema/Glove


In [3]:
import pandas as pd, numpy as np
import random
import csv, xlrd
import matplotlib.pyplot as plt 
%matplotlib inline
import re
import nltk
from nltk.corpus import stopwords
# nltk.download('punkt')
import string
import operator
import julia
import heapq
import graphviz
from collections import Counter
from pprint import pprint

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import tree
from sklearn import metrics
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import KFold
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV

In [4]:
wordgroups = list(pd.read_excel('wordgroups.xlsx')['Word Groupings'])

In [5]:
#set the seed
s =5

In [6]:
#Define Lasso as a function
def lasso(tolerance, verbose, test_X, test_y, train_X, train_y):
    tuned_parameters =[{'tol': np.arange(0.0001, tolerance + 0.01, 0.01)}]
    clf = GridSearchCV(LogisticRegression(penalty = 'l1',max_iter=1000, random_state=100), tuned_parameters, cv = 10, iid=True, scoring = score)
    clf.fit(train_X, train_y)
    print (clf.best_params_) 
    
    
    pred = clf.predict(test_X)
    proba = clf.predict_proba(test_X)[:,1]
    if verbose:
        print(classification_report(test_y, pred)) 
        features= abs(clf.best_estimator_.coef_[0]) > 0.3
        coef = pd.DataFrame()
        coef['word'] = test_X.columns[features]
        coef['coef'] = clf.best_estimator_.coef_[0,features]
        #df_c = pd.concat([Test.coef_[0,features].reset_index(drop=True), X_test.columns[features]], axis=1)
        print(coef.sort_values('coef', ascending= False))
    return proba

# Statistics Information

In [34]:
os.chdir(os.path.expanduser('~/Dropbox (Partners HealthCare)/MVP/Radiology NLP Project/Radiology Report Labeling/Stroke Labeling/Labeled Reports/NLP_Manuscript'))
labeled = pd.read_csv("Compiled_Reports_Stroke_Randomized_for_Kappas.csv")
labeled = labeled[['Report_Text']]
print(labeled.shape)

# replace whitespace with space
labeled['Report_Text'] = labeled['Report_Text'].apply(lambda text: ' '.join(text.split()))

# for x in labeled[labeled.EMPI == 105035731].Report_Text:
#     print(x)

# Remove header parts
# sum(['-'*78 in x or 'HISTORY:' in x or 'REPORT' in x for x in labeled.Report_Text])/len(labeled.Report_Text)
labeled['Report_Text'] = labeled.Report_Text.apply(lambda text: re.split('-'*78, text, 1)[-1])
labeled['Report_Text'] = labeled.Report_Text.apply(lambda text: re.split('HISTORY:', text, 1)[-1])
labeled['Report_Text'] = labeled.Report_Text.apply(lambda text: re.split('REPORT ', text, 1)[-1])
labeled['Report_Text'] = labeled.Report_Text.apply(lambda text: re.split('REPORT:', text, 1)[-1])

# Remove footer parts
labeled['Report_Text'] = labeled.Report_Text.apply(lambda text: 
                                                   re.split('electronically signed by:', text, flags=re.IGNORECASE)[0])
labeled['Report_Text'] = labeled.Report_Text.apply(lambda text: 
                                                   ''.join(re.split('i, the teaching physician, have reviewed the images and agree with the report as written', text, flags=re.IGNORECASE)))
labeled['Report_Text'] = labeled.Report_Text.apply(lambda text: 
                                                   re.split('radiologists: signatures:', text, flags=re.IGNORECASE)[0])
labeled['Report_Text'] = labeled.Report_Text.apply(lambda text: 
                                                   re.split('providers: signatures:', text, flags=re.IGNORECASE)[0])
labeled['Report_Text'] = labeled.Report_Text.apply(lambda text: 
                                                   re.split('findings were discussed on', text, flags=re.IGNORECASE)[0])
labeled['Report_Text'] = labeled.Report_Text.apply(lambda text: 
                                                   re.split('this report was electronically signed by', text, flags=re.IGNORECASE)[0])

# Remove reference texts =====
labeled['Report_Text'] = labeled.Report_Text.apply(lambda text: ''.join([x for i,x in enumerate(text.split('='*34)) if i != 1]))

## Get just IMPRESSION, convert both to lower
labeled['impression'] = labeled.Report_Text.apply(lambda text: text.split('IMPRESSION:')[0])
labeled['impression'] = labeled.impression.apply(lambda text: text.lower())
labeled['Report_Text'] = labeled.Report_Text.apply(lambda text: text.lower())
# Replace ngrams in Report_Text & IMPRESSION with their units
for group in wordgroups:
    labeled['impression'] = labeled.impression.apply(lambda text: text.replace(group, ''.join(group.split())))
    labeled['Report_Text'] = labeled.Report_Text.apply(lambda text: text.replace(group, ''.join(group.split())))

# CHECK WITH CHARLENE THAT IMPRESSIONS SHOULD NOT BE IDENTICAL AND SAME DAY
# TODO: later pull out just the date and just the time field, check for dupes on those
labeled = labeled.reset_index(drop=True)
print(labeled.shape)

(1359, 1)


In [55]:
labeled['Report_Text'].str.split().str.len().mean()
labeled['Report_Text'].str.split().str.len().median()
labeled['Report_Text'].str.strip().str.len().median()
labeled['Report_Text'].str.strip().str.len().mean()

1903.0669610007358

# Stroke

In [199]:
#First we need the previous models that we used for training with Lasso
os.chdir(os.path.expanduser("~/Dropbox (Partners HealthCare)/MVP/RPDR/GloVe/"))

num_seeds = 1
for technique in ['bow']:
    for classifier in ['logreg']:
#    for classifier in ['knn', 'logreg']:
        print('='*10, 'Results for', technique, classifier, '='*10)
        auc = 0
        for seed in range(1, num_seeds+1):
            print('Results for seed',s)
            # X = featurize(labeled_ct, technique, impression_only=False, d=200)
            # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed)
            X_train = pd.read_csv("clean_data/stroke/X_train_" + technique + '_' + str(s) + '.csv')
            X_test = pd.read_csv("clean_data/stroke/X_test_" + technique + '_' + str(s) + '.csv')
            y_train = pd.read_csv("clean_data/stroke/y_train_" + technique + '_' + str(s) + '.csv', header=None)
            y_test = pd.read_csv("clean_data/stroke/y_test_" + technique + '_' + str(s) + '.csv', header=None)
            y_train = y_train.iloc[:,0]
            y_test = y_test.iloc[:,0]
            
            tuned_parameters =[{'tol': np.arange(0.0001, 0.1 + 0.01, 0.01)}]
            clf = GridSearchCV(LogisticRegression(penalty = 'l1',max_iter=1000, random_state=100), 
                               tuned_parameters, cv = 10, iid=True, scoring = 'roc_auc')
            clf.fit(X_train, y_train)
            
#             tolerance = 0.1
#             score = 'roc_auc'
#             y_proba = lasso(tolerance, verbose, X_test, y_test, X_train, y_train)

            y_pred = clf.predict(X_test)
            y_proba = clf.predict_proba(X_test)[:,1]
            print(roc_auc_score(y_test, y_proba))
#             ouput_file.write(technique + ',' + classifier + ',' + str(seed) + ',' + str(roc_auc_score(y_test, y_proba)) + '\n')
            auc = auc + roc_auc_score(y_test, y_proba)
        print(technique, classifier, 'Average AUC:', auc/num_seeds)

========== Results for bow logreg ==========
Results for seed 5


/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

0.9550492610837439
bow logreg Average AUC: 0.9550492610837439


In [200]:
# Read the reports
os.chdir(os.path.expanduser("~/Dropbox (Partners HealthCare)/MVP/BMC External Validation"))
df = pd.read_csv("revision_labeled_reports_BMC.csv")
labeled.shape
labeled = df

In [201]:
labeled = labeled[['EMPI','Report_Number','Report_Date_Time','Report_Text']]
print(labeled.shape)

# Keep only those with IMPRESSIONS
labeled = labeled.iloc[[x for x in range(labeled.shape[0]) if 'IMPRESSION:' in labeled.Report_Text.iloc[x]]]
print(labeled.shape)

# replace whitespace with space
labeled['Report_Text'] = labeled['Report_Text'].apply(lambda text: ' '.join(text.split()))

# for x in labeled[labeled.EMPI == 105035731].Report_Text:
#     print(x)

# Remove header parts
# sum(['-'*78 in x or 'HISTORY:' in x or 'REPORT' in x for x in labeled.Report_Text])/len(labeled.Report_Text)
labeled['Report_Text'] = labeled.Report_Text.apply(lambda text: re.split('-'*78, text, 1)[-1])
labeled['Report_Text'] = labeled.Report_Text.apply(lambda text: re.split('HISTORY:', text, 1)[-1])
labeled['Report_Text'] = labeled.Report_Text.apply(lambda text: re.split('REPORT ', text, 1)[-1])
labeled['Report_Text'] = labeled.Report_Text.apply(lambda text: re.split('REPORT:', text, 1)[-1])

# Remove footer parts
labeled['Report_Text'] = labeled.Report_Text.apply(lambda text: 
                                                   re.split('electronically signed by:', text, flags=re.IGNORECASE)[0])
labeled['Report_Text'] = labeled.Report_Text.apply(lambda text: 
                                                   ''.join(re.split('i, the teaching physician, have reviewed the images and agree with the report as written', text, flags=re.IGNORECASE)))
labeled['Report_Text'] = labeled.Report_Text.apply(lambda text: 
                                                   re.split('radiologists: signatures:', text, flags=re.IGNORECASE)[0])
labeled['Report_Text'] = labeled.Report_Text.apply(lambda text: 
                                                   re.split('providers: signatures:', text, flags=re.IGNORECASE)[0])
labeled['Report_Text'] = labeled.Report_Text.apply(lambda text: 
                                                   re.split('findings were discussed on', text, flags=re.IGNORECASE)[0])
labeled['Report_Text'] = labeled.Report_Text.apply(lambda text: 
                                                   re.split('this report was electronically signed by', text, flags=re.IGNORECASE)[0])

# Remove reference texts =====
labeled['Report_Text'] = labeled.Report_Text.apply(lambda text: ''.join([x for i,x in enumerate(text.split('='*34)) if i != 1]))


# # Get just IMPRESSION, convert both to lower
# labeled['impression'] = labeled.Report_Text.apply(lambda text: text.split('IMPRESSION:')[1])
# labeled['impression'] = labeled.impression.apply(lambda text: text.lower())
labeled['Report_Text'] = labeled.Report_Text.apply(lambda text: text.lower())

# Replace ngrams in Report_Text & IMPRESSION with their units
for group in wordgroups:
#     labeled['impression'] = labeled.impression.apply(lambda text: text.replace(group, ''.join(group.split())))
    labeled['Report_Text'] = labeled.Report_Text.apply(lambda text: text.replace(group, ''.join(group.split())))

# CHECK WITH CHARLENE THAT IMPRESSIONS SHOULD NOT BE IDENTICAL AND SAME DAY
# TODO: later pull out just the date and just the time field, check for dupes on those
labeled = labeled.drop_duplicates(subset=['EMPI','Report_Date_Time','Report_Text'])
labeled = labeled.reset_index(drop=True)
print(labeled.shape)

(500, 4)
(500, 4)
(500, 4)


In [202]:
# BOW
vectorizer = CountVectorizer(min_df=2)
features = vectorizer.fit_transform(labeled.Report_Text).todense() 
vocab_counts = vectorizer.vocabulary_
feature_names = vectorizer.get_feature_names()
#Each one of the elements of the table 
X_all = pd.DataFrame(features, columns = feature_names)
#Now every row corresponds to a document and we have a vector of words that counts just the frequency in the matrix.
X_all.head()

,00,01,02,03,04,05,06,07,08,09,...,worsened,worsening,worst,would,xa0,year,yesterday,yo,zohaib,zygomatic
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [203]:
X_final = X_all[[x for x in X_all.columns if x in X_train.columns]]
missing = [x for x in X_train.columns if x not in X_final.columns]
extra = pd.DataFrame(0, index=np.arange(X_final.shape[0]), columns=missing)
X_final = pd.concat([X_final,extra], axis=1)
X_final = X_final.reindex(sorted(X_final.columns), axis=1)
pred = clf.predict_proba(X_final)
df['Stroke_preds'] = pred[:,0]

# Location

In [204]:
os.chdir(os.path.expanduser("~/Dropbox (Partners HealthCare)/MVP/RPDR/GloVe/"))

num_seeds = 1
for technique in ['bow']:
    for classifier in ['logreg']:
#    for classifier in ['knn', 'logreg']:
        print('='*10, 'Results for', technique, classifier, '='*10)
        auc = 0
        for seed in range(1, num_seeds+1):
            print('Results for seed',s)
            # X = featurize(labeled_ct, technique, impression_only=False, d=200)
            # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed)
            X_train = pd.read_csv("clean_data/location/X_train_" + technique + '_' + str(s) + '.csv')
            X_test = pd.read_csv("clean_data/location/X_test_" + technique + '_' + str(s) + '.csv')
            y_train = pd.read_csv("clean_data/location/y_train_" + technique + '_' + str(s) + '.csv', header=None)
            y_test = pd.read_csv("clean_data/location/y_test_" + technique + '_' + str(s) + '.csv', header=None)
            y_train = y_train.iloc[:,0]
            y_test = y_test.iloc[:,0]
            
            tuned_parameters =[{'tol': np.arange(0.0001, 0.1 + 0.01, 0.01)}]
            clf = GridSearchCV(LogisticRegression(penalty = 'l1',max_iter=1000, random_state=100), 
                               tuned_parameters, cv = 10, iid=True, scoring = 'roc_auc')
            clf.fit(X_train, y_train)
            
#             tolerance = 0.1
#             score = 'roc_auc'
#             y_proba = lasso(tolerance, verbose, X_test, y_test, X_train, y_train)

            y_pred = clf.predict(X_test)
            y_proba = clf.predict_proba(X_test)[:,1]
            print(roc_auc_score(y_test, y_proba))
#             ouput_file.write(technique + ',' + classifier + ',' + str(seed) + ',' + str(roc_auc_score(y_test, y_proba)) + '\n')
            auc = auc + roc_auc_score(y_test, y_proba)
        print(technique, classifier, 'Average AUC:', auc/num_seeds)

========== Results for bow logreg ==========
Results for seed 5


/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

0.964053199404762
bow logreg Average AUC: 0.964053199404762


/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [205]:
# BOW
vectorizer = CountVectorizer(min_df=2)
features = vectorizer.fit_transform(labeled.Report_Text).todense() 
vocab_counts = vectorizer.vocabulary_
feature_names = vectorizer.get_feature_names()
#Each one of the elements of the table 
X_all = pd.DataFrame(features, columns = feature_names)
#Now every row corresponds to a document and we have a vector of words that counts just the frequency in the matrix.
X_all.head()
X_final = X_all[[x for x in X_all.columns if x in X_train.columns]]
missing = [x for x in X_train.columns if x not in X_final.columns]
extra = pd.DataFrame(0, index=np.arange(X_final.shape[0]), columns=missing)
X_final = pd.concat([X_final,extra], axis=1)
X_final = X_final.reindex(sorted(X_final.columns), axis=1)
location = clf.predict_proba(X_final)
df['Location_preds'] = location[:,0]

# Acuity

In [206]:
os.chdir(os.path.expanduser("~/Dropbox (Partners HealthCare)/MVP/RPDR/GloVe/"))

num_seeds = 1
for technique in ['bow']:
    for classifier in ['logreg']:
#    for classifier in ['knn', 'logreg']:
        print('='*10, 'Results for', technique, classifier, '='*10)
        auc = 0
        for seed in range(1, num_seeds+1):
            print('Results for seed',s)
            # X = featurize(labeled_ct, technique, impression_only=False, d=200)
            # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed)
            X_train = pd.read_csv("clean_data/acuity/X_train_" + technique + '_' + str(s) + '.csv')
            X_test = pd.read_csv("clean_data/acuity/X_test_" + technique + '_' + str(s) + '.csv')
            y_train = pd.read_csv("clean_data/acuity/y_train_" + technique + '_' + str(s) + '.csv', header=None)
            y_test = pd.read_csv("clean_data/acuity/y_test_" + technique + '_' + str(s) + '.csv', header=None)
            y_train = y_train.iloc[:,0]
            y_test = y_test.iloc[:,0]
            
            tuned_parameters =[{'tol': np.arange(0.0001, 0.1 + 0.01, 0.01)}]
            clf = GridSearchCV(LogisticRegression(penalty = 'l1',max_iter=1000, random_state=100), 
                               tuned_parameters, cv = 10, iid=True, scoring = 'roc_auc')
            clf.fit(X_train, y_train)
            
#             tolerance = 0.1
#             score = 'roc_auc'
#             y_proba = lasso(tolerance, verbose, X_test, y_test, X_train, y_train)

            y_pred = clf.predict(X_test)
            y_proba = clf.predict_proba(X_test)[:,1]
            print(roc_auc_score(y_test, y_proba))
#             ouput_file.write(technique + ',' + classifier + ',' + str(seed) + ',' + str(roc_auc_score(y_test, y_proba)) + '\n')
            auc = auc + roc_auc_score(y_test, y_proba)
        print(technique, classifier, 'Average AUC:', auc/num_seeds)

========== Results for bow logreg ==========
Results for seed 5


/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

0.9001465201465202
bow logreg Average AUC: 0.9001465201465202


/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [207]:
# BOW
vectorizer = CountVectorizer(min_df=2)
features = vectorizer.fit_transform(labeled.Report_Text).todense() 
vocab_counts = vectorizer.vocabulary_
feature_names = vectorizer.get_feature_names()
#Each one of the elements of the table 
X_all = pd.DataFrame(features, columns = feature_names)
#Now every row corresponds to a document and we have a vector of words that counts just the frequency in the matrix.
X_all.head()
X_final = X_all[[x for x in X_all.columns if x in X_train.columns]]
missing = [x for x in X_train.columns if x not in X_final.columns]
extra = pd.DataFrame(0, index=np.arange(X_final.shape[0]), columns=missing)
X_final = pd.concat([X_final,extra], axis=1)
X_final = X_final.reindex(sorted(X_final.columns), axis=1)
acute = clf.predict_proba(X_final)
df['Acuity_preds'] = acute[:,0]

In [208]:
df.head()
df.to_csv('~/Dropbox (Partners HealthCare)/MVP/BMC External Validation/results/log_BMC_reports'+ '_' + str(s)+'.csv', index=False)

In [209]:
# Find which are the unique types of reports that are available
# reports.Report_Description.unique()
# relevant_report_types = ['CT HEAD WO CONTRAST',
#                          'MRI BRAIN', 
#                          'MRA HEAD AND NECK WITHOUT CONTRAST(STROKE)',
#                          'MRA HEAD WITHOUT CONTRAST', 
#                          'CT ANGIOGRAM HEAD W WO CONTRAST',
#                          'CT HEAD W CONTRAST', 
#                          'CT ANGIOGRAM HEAD AND NECK W W',
#                          'MRA HEAD W AND WO CONTRAST', 
#                          'CT HEAD W WO CONTRAST']

# #Keep only the relevant columns
# reports = reports[['Report ID','ID','proc_name','HOSP_ADMSN_TIME','HOSP_DISCH_TIME','SPECIMN_TAKEN_date','NOTE.TEXT']]
# reports = reports.rename(columns={"Report ID": "Report_Number", "ID": "EMPI","proc_name":"Report_Description","SPECIMN_TAKEN_date":"Report_Date_Time","NOTE.TEXT":"Report_Text"})
# print(reports.shape)

# # Keep only those with IMPRESSIONS
# reports = reports.iloc[[x for x in range(reports.shape[0]) if 'IMPRESSION:' in reports.Report_Text.iloc[x]]]
# print(labeled.shape)

# #Remove duplicates
# reports = reports.drop_duplicates(subset=None, keep='first', inplace=False)
# print(reports.shape)
# reports.head()

# reports = reports[reports.Report_Description.isin(relevant_report_types)]
# reports.shape

# reports = reports.sample(n=500, replace=False, random_state=1)
# reports.to_csv("1.21.20processed_epic_reports_n_500.csv", index=False)